# IMPORTING LIBRARIES

In [276]:
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option("display.max_colwidth", None)
import ast
import requests
import dask
import dask.dataframe as dd
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors
import numpy as np
import pickle
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import sigmoid_kernel

# 1. PROBLEM STATEMENT

#### Most users have trouble searching for movies that interest them and people dont have the time to scroll though different movies and see whether that movie interests them as there are tons of movies on the internet. Developing a system which provides the users with relevant movies which might interest them based on their taste and taste of people similar to them will help them in slecting movies better and faster. Plus users will tend to use more of the platform which provides better recommendations.

# 2. LOADING DATASET

In [4]:
movies5k = pd.read_csv("data/movie5k.csv")

In [25]:
links = pd.read_csv("data/links.csv")

In [309]:
ratings = pd.read_csv("data/ratings.csv")

# 3. PREPROCESSING

### MOVIES5K DATAFRAME

In [5]:
movies5k.head(1)

,color,director_name,num_critic_for_reviews,duration,director_facebook_likes,actor_3_facebook_likes,actor_2_name,actor_1_facebook_likes,gross,genres,actor_1_name,movie_title,num_voted_users,cast_total_facebook_likes,actor_3_name,facenumber_in_poster,plot_keywords,movie_imdb_link,num_user_for_reviews,language,country,content_rating,budget,title_year,actor_2_facebook_likes,imdb_score,aspect_ratio,movie_facebook_likes
0,Color,James Cameron,723.0,178.0,0.0,855.0,Joel David Moore,1000.0,760505847.0,Action|Adventure|Fantasy|Sci-Fi,CCH Pounder,Avatar,886204,4834,Wes Studi,0.0,avatar|future|marine|native|paraplegic,http://www.imdb.com/title/tt0499549/?ref_=fn_tt_tt_1,3054.0,English,USA,PG-13,237000000.0,2009.0,936.0,7.9,1.78,33000


In [6]:
movies5k.isna().sum()

color                         19
director_name                104
num_critic_for_reviews        50
duration                      15
director_facebook_likes      104
actor_3_facebook_likes        23
actor_2_name                  13
actor_1_facebook_likes         7
gross                        884
genres                         0
actor_1_name                   7
movie_title                    0
num_voted_users                0
cast_total_facebook_likes      0
actor_3_name                  23
facenumber_in_poster          13
plot_keywords                153
movie_imdb_link                0
num_user_for_reviews          21
language                      12
country                        5
content_rating               303
budget                       492
title_year                   108
actor_2_facebook_likes        13
imdb_score                     0
aspect_ratio                 329
movie_facebook_likes           0
dtype: int64

In [7]:
# SELECTING USEFULCOLUMNS
movies5k = movies5k.drop(
    [
        'color',
        'num_critic_for_reviews',
        'duration',
        'director_facebook_likes',
        'actor_3_facebook_likes',
        'actor_1_facebook_likes',
        'gross',
        'num_voted_users',
        'cast_total_facebook_likes',
        'facenumber_in_poster',
        'num_user_for_reviews',
        'language',
        'country',
        'content_rating',
        'budget',
        'actor_2_facebook_likes',
        'aspect_ratio',
        'movie_facebook_likes',
        'title_year',
        'imdb_score'
    ],
    axis = 1
)

In [9]:
# GETTING IMDB ID
movies5k['imdb_id'] = movies5k['movie_imdb_link'].apply(lambda x: float(x.split('tt')[2].split("/")[0]))
movies5k = movies5k.drop('movie_imdb_link', axis=1)

In [10]:
movies5k.head(1)

,director_name,actor_2_name,genres,actor_1_name,movie_title,actor_3_name,plot_keywords,imdb_id
0,James Cameron,Joel David Moore,Action|Adventure|Fantasy|Sci-Fi,CCH Pounder,Avatar,Wes Studi,avatar|future|marine|native|paraplegic,499549.0


In [11]:
movies5k.isna().sum()

director_name    104
actor_2_name      13
genres             0
actor_1_name       7
movie_title        0
actor_3_name      23
plot_keywords    153
imdb_id            0
dtype: int64

In [12]:
movies5k.dtypes

director_name     object
actor_2_name      object
genres            object
actor_1_name      object
movie_title       object
actor_3_name      object
plot_keywords     object
imdb_id          float64
dtype: object

### LINKS DATAFRAME

In [26]:
links.head(1)

,movieId,imdbId,tmdbId
0,1,114709,862.0


In [27]:
links.dtypes

movieId      int64
imdbId       int64
tmdbId     float64
dtype: object

In [28]:
links.isna().sum()

movieId      0
imdbId       0
tmdbId     219
dtype: int64

In [35]:
# FILLING NULL tmdb_id USING TMDB API
def fillnulltmdbid(x):
    imdbid = '0' * (7 - len(str(int(x)))) + str(int(x))
    URL = "https://api.themoviedb.org/3/find/tt" +imdbid+ "?api_key=142354a4b290b79eb450bc86ab04c9a4&language=en-US&external_source=imdb_id"
    r = requests.get(url = URL).json()
    try:
        if r['movie_results'][0]['id'] != '':
             return r['movie_results'][0]['id']
        else:
            return None
    except:
        return None

dask_links = dd.from_pandas(links, npartitions=1)
links.loc[pd.isna(links['tmdbId']), 'tmdbId'] = dask_links.loc[dd.isna(dask_links['tmdbId']), 'imdbId'].apply(lambda x : fillnulltmdbid(x), meta='float64').compute()

In [38]:
# CONVERTING TO FLOAT
links['imdbId'] = links['imdbId'].astype('float64')
links['movieId'] = links['movieId'].astype('float64')

In [39]:
# RENAMING COLUMNS
links = links.rename({'tmdbId': 'tmdb_id', 'imdbId': 'imdb_id', 'movieId': 'movie_id'}, axis=1)

In [40]:
links.isna().sum()

movie_id      0
imdb_id       0
tmdb_id     113
dtype: int64

In [41]:
links.dtypes

movie_id    float64
imdb_id     float64
tmdb_id     float64
dtype: object

In [42]:
links.head(1)

,movie_id,imdb_id,tmdb_id
0,1.0,114709.0,862.0


### MERGE DATAFRAMES

In [193]:
# MERGEING movies5k AND links
merge = movies5k.merge(links, left_on='imdb_id', right_on='imdb_id', how='left')

In [194]:
# REMOVING TV SHOWS
def gettv(x):
    imdbid = '0' * (7 - len(str(int(x)))) + str(int(x))
    URL = "https://api.themoviedb.org/3/find/tt" +imdbid+ "?api_key=142354a4b290b79eb450bc86ab04c9a4&language=en-US&external_source=imdb_id"
    r = requests.get(url = URL).json()
    if len(r['tv_results']) != 0:
        return x
    else:
        pass
dask_merge = dd.from_pandas(merge, npartitions=10)
imdb = dask_merge['imdb_id'].apply(lambda x : gettv(x), meta='float64').compute()

In [195]:
merge = merge[~merge['imdb_id'].isin(imdb)]

In [199]:
# FILL NULL TMDB ID
def gettv(x):
    imdbid = '0' * (7 - len(str(int(x)))) + str(int(x))
    URL = "https://api.themoviedb.org/3/find/tt" +imdbid+ "?api_key=142354a4b290b79eb450bc86ab04c9a4&language=en-US&external_source=imdb_id"
    r = requests.get(url = URL).json()
    try:
        return float(r['movie_results'][0]['id'])
    except:
        return None

dask_merge = dd.from_pandas(merge, npartitions=5) 
ans = dask_merge.loc[dd.isna(dask_merge['tmdb_id']), 'imdb_id'].apply(lambda x : gettv(x), meta='float64').compute()

In [200]:
merge.loc[pd.isna(merge['tmdb_id']), 'tmdb_id'] = ans

In [201]:
# FILL NULL TMDB ID
def get(x):
    URL = 'https://api.themoviedb.org/3/search/movie?api_key=142354a4b290b79eb450bc86ab04c9a4&query=' + x
    r = requests.get(url = URL).json()
    try:
        return float(r['results'][0]['id'])
    except:
        return None

dask_merge = dd.from_pandas(merge, npartitions=5)
ans = dask_merge.loc[dd.isna(dask_merge['tmdb_id']), 'movie_title'].apply(lambda x: get(x), meta='float64').compute()

In [202]:
merge.loc[pd.isna(merge['tmdb_id']), 'tmdb_id'] = ans

In [203]:
merge.isna().sum()

director_name      3
actor_2_name      12
genres             0
actor_1_name       7
movie_title        0
actor_3_name      21
plot_keywords    145
imdb_id            0
movie_id         220
tmdb_id            6
dtype: int64

In [204]:
# REMOVING NULL TMDBID
merge = merge[~pd.isna(merge['tmdb_id'])]

In [205]:
merge.isna().sum()

director_name      2
actor_2_name      11
genres             0
actor_1_name       6
movie_title        0
actor_3_name      19
plot_keywords    141
imdb_id            0
movie_id         214
tmdb_id            0
dtype: int64

In [209]:
# GETTING cast
def fillnull(x):
    URL = "https://api.themoviedb.org/3/movie/"+str(int(x))+"/credits?api_key=142354a4b290b79eb450bc86ab04c9a4&language=en-US"
    r = requests.get(url = URL).json()
    if 'cast' in r.keys():
        if len(r['cast']) != 0:
            return r['cast']
        else:
            return None
    else:
        return None

dask_merge = dd.from_pandas(merge, npartitions=10)
ans = dask_merge['tmdb_id'].apply(lambda x: fillnull(x), meta='object').compute()

In [212]:
merge['cast'] = ans

In [213]:
# GET overview
def get(x):
    URL = "https://api.themoviedb.org/3/movie/" +str(int(x))+ "?api_key=142354a4b290b79eb450bc86ab04c9a4&language=en-US"
    r = requests.get(url = URL).json()
    try:
        return r['overview']
    except:
        return None

dask_merge = dd.from_pandas(merge, npartitions=10)
ans = dask_merge['tmdb_id'].apply(lambda x: get(x), meta='object').compute()

In [214]:
merge['overview'] = ans

In [215]:
merge.isna().sum()

director_name      2
actor_2_name      11
genres             0
actor_1_name       6
movie_title        0
actor_3_name      19
plot_keywords    141
imdb_id            0
movie_id         214
tmdb_id            0
cast              28
overview           5
dtype: int64

In [216]:
merge.dtypes

director_name     object
actor_2_name      object
genres            object
actor_1_name      object
movie_title       object
actor_3_name      object
plot_keywords     object
imdb_id          float64
movie_id         float64
tmdb_id          float64
cast              object
overview          object
dtype: object

In [217]:
# GETTING PROPER FORMAT
merge['genres'] = merge['genres'].apply(lambda x: " ".join(x.split('|')))
merge['plot_keywords'] = merge['plot_keywords'].apply(lambda x: " ".join(str(x).split('|')))

In [219]:
# GETTING PROPER FORMAT of cast
def conv(x):
    try:
        return " ".join([i['name'] for i in x])
    except:
        return "unknown"
        
merge['cast_new'] = merge['cast'].apply(lambda x: conv(x))

In [226]:
# FILLING NULL
for i in ['director_name', 'actor_1_name', 'actor_2_name', 'actor_3_name', 'overview', 'cast']:
    merge[i] = merge[i].fillna('unknown')

In [228]:
merge.isna().sum()

director_name      0
actor_2_name       0
genres             0
actor_1_name       0
movie_title        0
actor_3_name       0
plot_keywords      0
imdb_id            0
movie_id         214
tmdb_id            0
cast               0
overview           0
cast_new           0
dtype: int64

In [234]:
# MAKING MAIN CONTENT DATA
merge['data'] = merge['movie_title'] +" "+ merge['director_name'] +" "+ merge['actor_1_name'] +" "+ merge['actor_2_name'] +" "+ merge['actor_3_name'] +" "+ merge['genres'] +" "+ merge['plot_keywords'] +" "+ merge['overview'] +" "+ merge['cast_new']

In [301]:
required = merge[['movie_title', 'imdb_id', 'movie_id', 'tmdb_id', 'data']]

In [302]:
required.loc[:,'tmdb_id'] = required.loc[:,'tmdb_id'].astype('int64')
required.loc[:,'imdb_id'] = required.loc[:,'imdb_id'].astype('int64')
required = required.drop_duplicates(subset=['movie_title'], keep="last")
required = required.drop_duplicates(subset=['tmdb_id'], keep="last")
required = required.drop_duplicates(subset=['imdb_id'], keep="last")
required['movie_title'] = required['movie_title'].apply(lambda x: x[:-1]).values

In [303]:
# SAVING REQUIRED DATAFRAME
required.to_csv('required.csv', index=False)

### RATINGS DATAFRAME

In [310]:
required = pd.read_csv('required.csv')

In [311]:
ratings.head(1)

,userId,movieId,rating,timestamp
0,1,110,1.0,1425941529


In [312]:
ratings.isna().sum()

userId       0
movieId      0
rating       0
timestamp    0
dtype: int64

In [313]:
ratings.dtypes

userId         int64
movieId        int64
rating       float64
timestamp      int64
dtype: object

In [314]:
ratings = ratings.drop('timestamp', axis=1)

In [315]:
# FILTERING RELEVANT MOVIES
ratings = ratings[ratings['movieId'].isin(required['movie_id'])]

In [316]:
# GETTING FREQUENCY OF EACH USER
user_freq_count = pd.DataFrame(ratings.groupby(['userId'])['rating'].count()).reset_index()
user_freq_count.columns = ['ID', 'COUNT']

In [317]:
# GETTING USERS WITH SPECIFIC FREQUENCY OF VOTES
freq = user_freq_count.loc[user_freq_count['COUNT']>500, 'ID']
# SPLITTING THE DF ACCORDINGLY
collaborative = ratings[ratings['userId'].isin(freq)]
collaborative = collaborative.pivot_table(index='movieId',columns='userId',values='rating').fillna(0)

In [318]:
# SHOW TOP
collaborative.head()

userId   46      229     231     332     429     483     533     567     \
movieId                                                                   
1           4.5     3.0     2.5     4.5     4.0     3.5     3.0     2.0   
10          0.0     0.0     3.0     0.0     2.5     3.5     4.0     0.0   
11          0.0     0.0     0.0     0.0     0.0     0.0     3.0     0.0   
14          0.0     0.0     0.0     0.0     0.0     0.0     0.0     0.0   
15          0.0     0.0     0.0     0.0     2.0     0.0     3.0     0.0   

userId   682     741     778     825     836     924     930     971     \
movieId                                                                   
1           4.0     4.5     3.0     3.5     3.5     3.0     3.5     4.5   
10          3.0     3.0     2.0     0.0     0.0     1.0     3.5     3.0   
11          3.0     0.0     0.0     0.0     0.0     0.0     0.0     0.0   
14          0.0     0.0     0.0     0.0     0.0     0.0     0.0     3.5   
15          0.0     3.0     0.0     0.0     0.0     0.0     0.0     0.0   

userId   1104    1136    1178    1243    1250    1316    1380    1484    \
movieId                                                                   
1           4.5     5.0     4.0     4.0     5.0     4.0     4.5     4.0   
10          3.5     4.0     3.5     0.0     0.0     3.5     0.0     0.0   
11          0.0     4.0     0.0     5.0     0.0     0.0     0.0     0.0   
14          0.0     0.0     0.0     0.0     0.0     0.0     0.0     0.0   
15          0.0     0.0     0.0     0.0     0.0     0.0     0.0     0.0   

userId   1652    1709    1802    1846    1859    1932    2022    2191    \
movieId                                                                   
1           5.0     2.5     3.0     3.5     4.0     3.5     4.0     3.0   
10          3.0     3.5     2.5     3.5     4.5     3.0     3.0     0.0   
11          0.0     0.0     0.0     0.0     0.0     0.0     0.0     0.0   
14          0.0     0.0     0.0     0.0     0.0     0.0     0.0     0.0   
15          0.0     0.0     0.0     0.0     0.0     0.0     0.0     3.0   

userId   2306    2313    2363    2489    2490    2520    2531    2547    \
movieId                                                                   
1           2.0     5.0     4.5     0.0     4.0     0.0     3.5     4.0   
10          4.0     0.0     4.0     0.5     4.0     0.0     3.0     0.0   
11          3.5     0.0     0.0     0.0     4.0     0.0     3.0     0.0   
14          0.0     3.0     0.0     0.0     4.0     0.0     3.5     0.0   
15          0.0     0.0     0.0     0.0     3.0     0.0     0.0     0.0   

userId   2555    2609    2627    2639    2642    2654    2885    2975    \
movieId                                                                   
1           3.5     4.0     4.0     3.5     0.0     3.5     3.0     4.5   
10          4.0     4.0     4.0     3.0     2.0     4.0     0.0     0.0   
11          0.0     0.0     0.0     0.0     3.0     0.0     0.0     0.0   
14          0.0     0.0     0.0     0.0     0.0     0.0     4.5     0.0   
15          0.0     0.0     0.0     0.0     0.0     0.0     0.0     0.0   

userId   3054    3140    3222    3437    3519    3586    3673    3694    \
movieId                                                                   
1           0.0     4.0     3.0     3.5     0.0     3.5     0.0     0.0   
10          4.0     4.0     0.0     4.0     0.0     0.0     4.0     3.0   
11          0.0     3.0     2.5     3.0     0.0     3.5     0.0     0.0   
14          0.0     0.0     0.0     3.0     0.0     0.0     0.0     0.0   
15          0.0     0.0     1.5     2.5     0.0     0.0     0.0     0.0   

userId   3740    3767    3778    3780    3950    4013    4095    4160    \
movieId                                                                   
1           4.0     4.0     5.0     3.0     3.0     4.0     4.0     5.0   
10          3.5     0.0     2.0     0.0     2.0     0.0     0.0     3.0   
11          0.0   

In [319]:
# SAVING COLLABORATIVE DATAFRAME
collaborative.to_csv('collaborative.csv')

# MODEL BUILDING

## COLLABORATIVE

In [320]:
collaborative = pd.read_csv('collaborative.csv')

In [321]:
collaborative.head()

movieId   46  229  231  332  429  483  533  567  682  741  778  825  836  \
0        1  4.5  3.0  2.5  4.5  4.0  3.5  3.0  2.0  4.0  4.5  3.0  3.5  3.5   
1       10  0.0  0.0  3.0  0.0  2.5  3.5  4.0  0.0  3.0  3.0  2.0  0.0  0.0   
2       11  0.0  0.0  0.0  0.0  0.0  0.0  3.0  0.0  3.0  0.0  0.0  0.0  0.0   
3       14  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
4       15  0.0  0.0  0.0  0.0  2.0  0.0  3.0  0.0  0.0  3.0  0.0  0.0  0.0   

   924  930  971  1104  1136  1178  1243  1250  1316  1380  1484  1652  1709  \
0  3.0  3.5  4.5   4.5   5.0   4.0   4.0   5.0   4.0   4.5   4.0   5.0   2.5   
1  1.0  3.5  3.0   3.5   4.0   3.5   0.0   0.0   3.5   0.0   0.0   3.0   3.5   
2  0.0  0.0  0.0   0.0   4.0   0.0   5.0   0.0   0.0   0.0   0.0   0.0   0.0   
3  0.0  0.0  3.5   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   
4  0.0  0.0  0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   

   1802  1846  1859  1932  2022  2191  2306  2313  2363  2489  2490  2520  \
0   3.0   3.5   4.0   3.5   4.0   3.0   2.0   5.0   4.5   0.0   4.0   0.0   
1   2.5   3.5   4.5   3.0   3.0   0.0   4.0   0.0   4.0   0.5   4.0   0.0   
2   0.0   0.0   0.0   0.0   0.0   0.0   3.5   0.0   0.0   0.0   4.0   0.0   
3   0.0   0.0   0.0   0.0   0.0   0.0   0.0   3.0   0.0   0.0   4.0   0.0   
4   0.0   0.0   0.0   0.0   0.0   3.0   0.0   0.0   0.0   0.0   3.0   0.0   

   2531  2547  2555  2609  2627  2639  2642  2654  2885  2975  3054  3140  \
0   3.5   4.0   3.5   4.0   4.0   3.5   0.0   3.5   3.0   4.5   0.0   4.0   
1   3.0   0.0   4.0   4.0   4.0   3.0   2.0   4.0   0.0   0.0   4.0   4.0   
2   3.0   0.0   0.0   0.0   0.0   0.0   3.0   0.0   0.0   0.0   0.0   3.0   
3   3.5   0.0   0.0   0.0   0.0   0.0   0.0   0.0   4.5   0.0   0.0   0.0   
4   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   

   3222  3437  3519  3586  3673  3694  3740  3767  3778  3780  3950  4013  \
0   3.0   3.5   0.0   3.5   0.0   0.0   4.0   4.0   5.0   3.0   3.0   4.0   
1   0.0   4.0   0.0   0.0   4.0   3.0   3.5   0.0   2.0   0.0   2.0   0.0   
2   2.5   3.0   0.0   3.5   0.0   0.0   0.0   0.0   0.0   5.0   3.0   0.0   
3   0.0   3.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   
4   1.5   2.5   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   

   4095  4160  4200  4219  4294  4323  4345  4351  4384  4387  4426  4450  \
0   4.0   5.0   3.5   5.0   5.0   3.0   2.5   4.0   3.5   4.5   0.0   1.0   
1   0.0   3.0   4.0   4.0   4.0   0.0   0.0   0.0   0.0   3.5   3.0   2.0   
2   2.0   0.0   0.0   5.0   0.0   4.0   0.0   0.0   0.0   0.0   3.0   0.0   
3   0.0   0.0   0.0   0.0   0.0   4.0   0.0   0.0   0.0   0.0   3.5   0.0   
4   0.0   0.0   0.0   0.0   3.0   0.0   0.0   0.0   0.0   0.0   1.0   0.0   

   4493  4496  4540  4579  4666  4785  4916  4932  4964  5102  5127  5148  \
0   4.0   0.0   4.5   4.0   3.0   3.5   3.5   2.5   5.0   3.0   4.0   3.0   
1   0.0   0.0   0.0   3.5   0.0   4.0   3.5   0.0   0.0   2.0   4.0   3.0   
2   3.0   0.0   0.0   0.0   3.5   3.5   0.0   0.0   4.0   0.0   0.0   0.0   
3   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   
4   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   1.0   0.0   0.0   

   5229  5243  5256  5300  5305  5332  5342  5378  5416  5515  5523  5620  \
0   0.0   0.0   4.0   0.0   2.5   2.5   4.0   4.0   4.5   4.0   5.0   5.0   
1   0.0   0.0   3.5   0.0   0.0   2.5   3.0   2.0   0.0   4.0   3.5   0.0   
2   3.0   3.0   3.5   0.0   0.0   0.0   0.0   2.0   5.0   0.0   0.0   0.0   
3   0.0   0.0   0.0   3.0   0.0   0.0   0.0   0.0   0.0   0.0   3.0   0.0   
4   0.0   0.0   0.0   0.0   0.0   4.0   0.0   0.0   0.0   3.5   0.0   0.0   

   5641  5662  5765  5811  5829  5875  5915  5936  5953  6042  6064  6074  \
0   3.0   3.0   3.0   3.0   2.5   2.5   4.0   3.5   4.5   3.5   2.5   0.5   
1   3.0   3.0   0.0   3.0   3.0   3.0   3.5   3.5   4.0   3.5   3.0   4.0   
2   0.0   0.0   0.0   3.0   2.5   0.0 

In [322]:
# MAKE CSR MATRIX
collaborative_matrix = csr_matrix(collaborative.drop('movieId', axis=1).values)

In [323]:
# MAKING KNN MODEL
model_knn = NearestNeighbors(metric = 'cosine', algorithm = 'brute').fit(collaborative_matrix)

In [324]:
def collaborative_rec():
    # GET RANDOM MOVIE
    query_index = np.random.choice(collaborative['movieId'])
    print(required.loc[required['movie_id'] == query_index, 'movie_title'].values)
    # FIND NEAREST MOVIES
    distances, indices = model_knn.kneighbors(collaborative[collaborative['movieId'] == query_index].drop('movieId', axis=1).values.reshape(1, -1), n_neighbors = 10)
    indices = list(indices[0])
    movie_id = collaborative.iloc[indices]['movieId'].values
    print(required[required['movie_id'].isin(movie_id)]['movie_title'].values)

In [325]:
collaborative_rec()

['Layer Cake']
['The Departed' 'Children of Men' 'Collateral' 'Sin City'
 'No Country for Old Men' 'Kiss Kiss Bang Bang' 'Casino Royale' 'Snatch'
 'Layer Cake' 'Lock, Stock and Two Smoking Barrels']


In [326]:
# SAVE MODEL
filename = 'model/COLL.sav'
pickle.dump(model_knn, open(filename, 'wb'))

In [327]:
# LOAD MODEL
COLL = pickle.load(open(filename, 'rb'))
query_index = np.random.choice(collaborative['movieId'])
print(required.loc[required['movie_id'] == query_index, 'movie_title'].values)
distances, indices = COLL.kneighbors(collaborative[collaborative['movieId'] == query_index].drop('movieId', axis=1).values.reshape(1, -1), n_neighbors = 10)
indices = list(indices[0])
indices = collaborative.iloc[indices]['movieId'].values
print(required[required['movie_id'].isin(indices)]['movie_title'].values)

['The Color of Freedom']
['The Color of Freedom' 'The Children of Huang Shi' 'Broken Horses'
 'Blood Done Sign My Name' 'Made in Dagenham' 'Two Lovers and a Bear'
 'A Shine of Rainbows' 'Enter the Dangerous Mind'
 "Gangster's Paradise: Jerusalema" 'Rockaway']


## CONTENT

In [328]:
required = required.drop_duplicates(subset='movie_title', keep="last")

In [329]:
# COMPUTE TFIDF 
v = TfidfVectorizer()
x = v.fit_transform(required['data'])

In [330]:
# SIGMOID KERNEL
sig = sigmoid_kernel(x, x)

In [331]:
indices = pd.Series(required.index, index=required['tmdb_id'].apply(lambda x: int(x))).drop_duplicates()
rev = pd.Series(required['tmdb_id'].apply(lambda x: int(x)), index=required.index).drop_duplicates()

In [332]:
def content_rec():
    # GET TMDB ID
    tmdb = int(np.random.choice(required['tmdb_id']))
    print(required.loc[required['tmdb_id'] == tmdb, 'movie_title'].values)
    idx = indices[tmdb]
    
    # GET PAIRWISE SIMILARITY SCORE 
    sig_scores = list(enumerate(sig[idx]))

    # SORTING MOVIES
    sig_scores = sorted(sig_scores, key=lambda x: x[1], reverse=True)

    # GET MOST SIMILAR MOVIES PAIRS
    sig_scores = sig_scores[1:15]

    # GET TMDB ID
    idx = [i[0] for i in sig_scores]
    tmdb = []
    for i in idx:
        try:
            tmdb.append(rev[i])
        except:
            pass

    # PRINT SIMILAR MOVIES
    print([required[required['tmdb_id'] == float(i)]['movie_title'].values[0] for i in tmdb])

In [333]:
content_rec()

['City of Angels']
['Frequency', "The Caveman's Valentine", 'Bangkok Dangerous', 'Adaptation.', 'Face/Off', 'Poseidon', 'Ghost Rider', 'Fantastic 4: Rise of the Silver Surfer', 'Astro Boy', 'The Exorcism of Emily Rose', 'Jack Ryan: Shadow Recruit', 'Changeling', 'The Croods', 'Critical Care']


In [334]:
# SAVE MODEL
filename = 'model/CONT.sav'
pickle.dump(sig, open(filename, 'wb'))

In [335]:
# LOAD MODEL
CONT = pickle.load(open(filename, 'rb'))
tmdb = int(np.random.choice(required['tmdb_id']))
print(required.loc[required['tmdb_id'] == tmdb, 'movie_title'].values)
idx = indices[tmdb]
sig_scores = list(enumerate(CONT[idx]))
sig_scores = sorted(sig_scores, key=lambda x: x[1], reverse=True)
sig_scores = sig_scores[1:15]
idx = [i[0] for i in sig_scores]
tmdb = []
for i in idx:
    try:
        tmdb.append(rev[i])
    except:
        pass
print([required[required['tmdb_id'] == float(i)]['movie_title'].values[0] for i in tmdb])

['The Legend of the Lone Ranger']
['The Lone Ranger', 'Lone Wolf McQuade', 'Man of the House', 'The Black Stallion', 'Texas Rangers', 'Dances with Wolves', 'American Outlaws', 'Butch Cassidy and the Sundance Kid', "The Legend of Hell's Gate: An American Conspiracy", 'Last Action Hero', 'The Tree of Life', 'Conan the Barbarian', 'The Assassination of Jesse James by the Coward Robert Ford', 'Red River']
